In [91]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [92]:

# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 256
n_head = 4
n_layer = 6
dropout = 0.0

In [117]:
file_path = "/home/max/Bureau/Python/IA_CG/NLP_projet_2/cleaned_merged_fairy_tales_without_eos.txt"

# Ouvrir et lire le fichier
with open(file_path, 'r', encoding='utf-8') as file:
    data = file.read()

# Calculer la longueur de 10% du texte
ten_percent_length = int(len(data) * 0.05)

# Extraire 10% du texte
text = data[:ten_percent_length]
print("length of dataset in characters: ", len(text))

print(text[:64])



length of dataset in characters:  1022784
The Happy Prince.
HIGH above the city, on a tall column, stood t


In [104]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("character set used in model training : ",chars)
print("The size of the vocab",vocab_size)

character set used in model training :  ['\n', ' ', '!', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”']
The size of the vocab 83


In [106]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("HIGH above the city, on a tall column, stood the statue of the Happy Prince."))
print(decode([0]))

[31, 32, 30, 31, 1, 52, 53, 66, 73, 56, 1, 71, 59, 56, 1, 54, 60, 71, 76, 7, 1, 66, 65, 1, 52, 1, 71, 52, 63, 63, 1, 54, 66, 63, 72, 64, 65, 7, 1, 70, 71, 66, 66, 55, 1, 71, 59, 56, 1, 70, 71, 52, 71, 72, 56, 1, 66, 57, 1, 71, 59, 56, 1, 31, 52, 67, 67, 76, 1, 39, 69, 60, 65, 54, 56, 9]




In [96]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1022784]) torch.int64
tensor([43, 59, 56,  1, 31, 52, 67, 67, 76,  1, 39, 69, 60, 65, 54, 56,  9,  0,
        31, 32, 30, 31,  1, 52, 53, 66, 73, 56,  1, 71, 59, 56,  1, 54, 60, 71,
        76,  7,  1, 66, 65,  1, 52,  1, 71, 52, 63, 63,  1, 54, 66, 63, 72, 64,
        65,  7,  1, 70, 71, 66, 66, 55,  1, 71, 59, 56,  1, 70, 71, 52, 71, 72,
        56,  1, 66, 57,  1, 71, 59, 56,  1, 31, 52, 67, 67, 76,  1, 39, 69, 60,
        65, 54, 56,  9,  1,  1, 31, 56,  1, 74, 52, 70,  1, 58, 60, 63, 55, 56,
        55,  1, 52, 63, 63,  1, 66, 73, 56, 69,  1, 74, 60, 71, 59,  1, 71, 59,
        60, 65,  1, 63, 56, 52, 73, 56, 70,  1, 66, 57,  1, 57, 60, 65, 56,  1,
        58, 66, 63, 55,  7,  1, 57, 66, 69,  1, 56, 76, 56, 70,  1, 59, 56,  1,
        59, 52, 55,  1, 71, 74, 66,  1, 53, 69, 60, 58, 59, 71,  1, 70, 52, 67,
        67, 59, 60, 69, 56, 70,  7,  1, 52, 65, 55,  1, 52,  1, 63, 52, 69, 58,
        56,  1, 69, 56, 55,  1, 69, 72, 53, 76,  1, 58, 63, 66, 74, 56, 55,  1,
      

In [97]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [98]:

train_data[:block_size+1]

tensor([43, 59, 56,  1, 31, 52, 67, 67, 76,  1, 39, 69, 60, 65, 54, 56,  9,  0,
        31, 32, 30, 31,  1, 52, 53, 66, 73, 56,  1, 71, 59, 56,  1, 54, 60, 71,
        76,  7,  1, 66, 65,  1, 52,  1, 71, 52, 63, 63,  1, 54, 66, 63, 72, 64,
        65,  7,  1, 70, 71, 66, 66, 55,  1, 71, 59])

In [99]:

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [100]:

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



4.793427 M parameters


step 0: train loss 4.6596, val loss 4.6598
step 100: train loss 2.2824, val loss 2.2891
step 200: train loss 2.0022, val loss 2.0275
step 300: train loss 1.8209, val loss 1.8678
step 400: train loss 1.7011, val loss 1.7538
step 500: train loss 1.6307, val loss 1.6987
step 600: train loss 1.5641, val loss 1.6385
step 700: train loss 1.5081, val loss 1.5957
step 800: train loss 1.4831, val loss 1.5761
step 900: train loss 1.4527, val loss 1.5504
step 1000: train loss 1.4172, val loss 1.5157
step 1100: train loss 1.3972, val loss 1.5070
step 1200: train loss 1.3748, val loss 1.4852
step 1300: train loss 1.3483, val loss 1.4748
step 1400: train loss 1.3368, val loss 1.4621
step 1500: train loss 1.3245, val loss 1.4522
step 1600: train loss 1.3035, val loss 1.4435
step 1700: train loss 1.2956, val loss 1.4305
step 1800: train loss 1.2825, val loss 1.4310
step 1900: train loss 1.2764, val loss 1.4254
step 2000: train loss 1.2636, val loss 1.4109
step 2100: train loss 1.2589, val loss 1.4196


In [105]:
#1.387
# generate from the model


start_tokens_ids = encode("HIGH above the city, on a tall column, stood the statue of the Happy Prince.") 
print ("start_tokens_ids = ",start_tokens_ids)

print(decode(m.generate(start_tokens_ids, max_new_tokens=2000)[0].tolist()))

start_tokens_ids =  [31, 32, 30, 31, 1, 52, 53, 66, 73, 56, 1, 71, 59, 56, 1, 54, 60, 71, 76, 7, 1, 66, 65, 1, 52, 1, 71, 52, 63, 63, 1, 54, 66, 63, 72, 64, 65, 7, 1, 70, 71, 66, 66, 55, 1, 71, 59, 56, 1, 70, 71, 52, 71, 72, 56, 1, 66, 57, 1, 71, 59, 56, 1, 31, 52, 67, 67, 76, 1, 39, 69, 60, 65, 54, 56, 9]
HIGH above the city, on a tall column, stood the statue of the Happy Prince.  One has nothing at at once we shall come to what her most before the King saves, to live on its head head at the Princess, the Vizite prepared with a bundle, and he soon found depting himself, he swam Ali:
“Here is that it is it,”
 said the old grapititude. “And there fly quite leagulorrow; but the handnest must what they were were considered, but the mind now rest her the Sheep said. “You have a fewolves descense!” said the handles. “Nice were from,” said he, “and It is too.  Here is as evil as let were forced to make as distrey they, at a beggar: it was even that procularly was that the one thickest was l

In [115]:
import torch.onnx

# Supposons que 'model' est votre modèle PyTorch et 'dummy_input' est un exemple d'entrée
dummy_input = torch.zeros(1, block_size, dtype=torch.long).to(device)  # block_size doit correspondre à votre modèle

torch.onnx.export(model, dummy_input, "model.onnx", opset_version=11, input_names=['input'], output_names=['output'], dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})


In [110]:
print("dummy_input = ",dummy_input)
context2 = torch.zeros((1, 1), dtype=torch.long, device=device)
print(context2)

dummy_input =  tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')
tensor([[0]], device='cuda:0')


In [116]:
# Créer un exemple d'entrée
# Supposons que vous utilisiez des entiers pour représenter des tokens
sequence_length = 64  # Longueur de la séquence attendue par le modèle
example_sequence = [64]  # Exemple de séquence d'entrée
padding_length = sequence_length - len(example_sequence)

# Appliquer le padding

input_sequence = example_sequence + [0] * padding_length

# Convertir en tenseur PyTorch
input_tensor = torch.tensor([input_sequence], dtype=torch.long).to(device)

# Passer l'entrée dans le modèle
with torch.no_grad():
    model.eval()
    output = model(input_tensor)

# Afficher la sortie
print(output)

(tensor([[[-4.9098,  3.7177, -0.0373,  ..., -1.8795, -3.1893, -2.1968],
         [-0.6648,  4.8009, -1.5613,  ..., -4.4058,  4.2152, -2.9461],
         [ 0.8904,  4.9534, -3.2890,  ..., -3.9753,  5.6588, -3.7643],
         ...,
         [ 2.5446,  4.7969, -3.2776,  ..., -4.3524,  6.6065, -4.8146],
         [ 2.7729,  4.7803, -3.3443,  ..., -4.2029,  6.9469, -4.5609],
         [ 2.7572,  4.8237, -3.3616,  ..., -4.2111,  6.8846, -4.5310]]],
       device='cuda:0'), None)
